In [ ]:
!pip install langchain openai chromadb tiktoken unstructured langchain_community streamlit
!npm install localtunnel




In [ ]:
%%writefile app.py

import os
import sys
import streamlit as st

from google.colab import userdata
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.vectorstores import Chroma
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import TextLoader
from langchain_core.chat_history import BaseChatMessageHistory
from langchain.prompts import ChatPromptTemplate, FewShotPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Set the API key for OpenAI
# set openai's api key in variable 'openai_api_key'

os.environ['OPENAI_API_KEY'] = openai_api_key

# Configuration for persistence
PERSIST = False

# Get the query from command line arguments if provided
query = None
if len(sys.argv) > 1:
    query = sys.argv[1]

### Construct retriever ###
loader = TextLoader("/content/data/appian_components_commented.txt")  # Adjust this path as needed
docs = loader.load()
#chunking document into smaller sections to fit context window
#chunk overlap for retaining context
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

if PERSIST:
    vectorstore.persist(directory="persist")

retriever = vectorstore.as_retriever()

### Contextualize question ###
contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    ChatOpenAI(model="gpt-4o"),
    retriever,
    contextualize_q_prompt
)

### Answer question ###
few_shot_examples_qa=[
    {
        "question": "Create a simple blood donation form in Appian SAIL Code",
        "answer": """
a!localVariables( local!donorFirstName,
 local!donorLastName,
 local!donorEmail,
 local!donorBloodType,
 local!donorAvailability,
 a!formLayout(
   label: "Blood Donation Application",
   contents: {{
     a!textField(
       label: "First Name",
       value: local!donorFirstName,
       saveInto: local!donorFirstName,
       required: true
     ),
     a!textField(
       label: "Last Name",
       value: local!donorLastName,
       saveInto: local!donorLastName,
       required: true
     ),
     a!textField(
       label: "Email Address",
       value: local!donorEmail,
       saveInto: local!donorEmail,
       required: true,
       validations: if(not(isnull(local!donorEmail)) and not(contains(local!donorEmail, "@")), "Please enter a valid email address")
     ),
     a!dropdownField(
       label: "Blood Type",
       placeholder: "Select Your Blood Type",
       choiceLabels: {{"A+", "A-", "B+", "B-", "AB+", "AB-", "O+", "O-"}},
       choiceValues: {{"A+", "A-", "B+", "B-", "AB+", "AB-", "O+", "O-"}},
       value: local!donorBloodType,
       saveInto: local!donorBloodType
     ),
     a!dateField(
       label: "Availability Date",
       instructions: "Select the date you are available to donate blood",
       value: local!donorAvailability,
       saveInto: local!donorAvailability,
       required: true
     ),
     a!buttonArrayLayout(
       buttons: {{
         a!buttonWidget(
           label: "Submit Application",
           style: "GHOST",
           saveInto: {{}}
         )
       }}
     )
   }}
 )
)
"""
    },
    {
        "question": "Generate Appian SAIL Code for a job interview form",
        "answer":"""
a!localVariables(
  local!firstName,
  local!lastName,
  local!email,
  local!phoneNumber,
  local!jobPosition,
  local!resume,
  local!coverLetter,
  a!formLayout(
    label: "Job Interview Form",
    contents: {{
      a!textField(
        label: "First Name",
        value: local!firstName,
        saveInto: local!firstName
      ),
      a!textField(
        label: "Last Name",
        value: local!lastName,
        saveInto: local!lastName
      ),
      a!textField(
        label: "Email",
        value: local!email,
        saveInto: local!email,
        validations: {{
          if(
            not(isnull(local!email)) and not(contains(local!email, "@")),
            "Please enter a valid email address"
          )
        }}
      ),
      a!textField(
        label: "Phone Number",
        value: local!phoneNumber,
        saveInto: local!phoneNumber
      ),
      a!dropdownField(
        label: "Job Position",
        placeholder: "--- Select Job Position ---",
        choiceLabels: {{"Software Engineer", "Product Manager", "Designer"}},
        choiceValues: {{"SE", "PM", "D"}},
        value: local!jobPosition,
        saveInto: local!jobPosition
      ),
      a!fileUploadField(
        label: "Cover Letter",
        value: local!coverLetter,
        saveInto: local!coverLetter
      ),
      a!buttonArrayLayout(
        buttons: {{
          a!buttonWidget(
            label: "Submit",
            style: "GHOST",
            saveInto: {{}}
          )
        }}
      )
    }}
  )
)
        """
    }
]

qa_examples = "\n\n".join(
    [f"Input:\n{example['question']}\nAnswer:\n{example['answer']}" for example in few_shot_examples_qa]
)

qa_system_prompt = f"""You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Generate only Appian SAIL Code in response to the prompts.
You will be expected to write SAIL code to generate User Interface for forms and websites.
Use only the components mentioned in document data.txt for code generations.
Rely on your knowledge of Appian SAIL code structure while generating code.
Always define variables at the start using local! variables.
Form layouts have to be on the same level as the declarations of the local variables. Here is an example: a!localVariables(local!firstName, local!lastName, local!email, a!formLayout() ) \
a!formLayout should not be encased by curly paranthesis, it should be on the same level as the local variables.
a!dropdownField needs a placeholder if the initial value is a local variable that is not given a default.
a!emailField does not exist, just use a!textField.
While defining a button, define buttonWidget inside buttonLayout.
Always use only the following three parameters inside buttonWidget: label, style, and saveInto.
Always set value of style in all buttonWidget to GHOST.
Always make sure all opening parentheses match closing parentheses.
if statements for validations only contain 2 parameters (condition, result), and does not contain a \"null\" as a last parameter. You don't need to include it. Here is an example: if(not(isnull(local!email)) and not(contains(local!email, \"@\")), \"Please enter a valid email address\").

Examples: {qa_examples}

Retrieved context:
{{context}}
"""

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(ChatOpenAI(model="gpt-4o"), qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

### Statefully manage chat history ###
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

# Streamlit app
st.title("Chatbot for Appian SAIL Code Generation")

if 'session_id' not in st.session_state:
    st.session_state.session_id = "abc123"

if 'chat_history' not in st.session_state:
    st.session_state.chat_history = []

# User input
query = st.text_input("Prompt", key="input_text")

if st.button("Submit"):
    if query:
        # Get the result from the chain
        result = conversational_rag_chain.invoke(
            {"input": f"{query}"},
            config={
                "configurable": {"session_id": st.session_state.session_id}
            },
        )
        # Update chat history
        st.session_state.chat_history.append((query, result['answer']))
        st.write(f"**You:** {query}")
        st.write("**Chatbot:**")
        st.code(f"{result['answer']}")

# Display chat history
st.write("## Chat History")
for q, a in st.session_state.chat_history:
    st.write(f"**You:** {q}")
    st.write(f"**Chatbot:**")
    st.code(f"{a}")


Writing app.py


In [ ]:
!streamlit run app.py &>/content/logs.txt & echo "copy this IP address for your 'Tunnel Password:"; curl ipv4.icanhazip.com
!npx localtunnel --port 8501

copy this IP address for your 'Tunnel Password:
34.86.6.130
your url is: https://some-terms-doubt.loca.lt
